In [1]:
import os
import sys
sys.path.insert(0, '..')
from lib.AirtableLib import AirtableData

In [2]:
personal_base_id = 'appYknCML4IMHe9DP'
personal_table_id = 'tblQcjtQ7kB3dJx1m'
airtable_api_key = os.environ.get('AIRTABLE_API_KEY')

In [4]:
personal_airtable = AirtableData(personal_base_id, personal_table_id, airtable_api_key)
personal_data = personal_airtable.get_data()
print(personal_data.shape)

(459, 62)


## Oura Columns Work

In [5]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Any, Union

#### Testing Oura API endpoints

In [6]:
base_url = 'https://api.ouraring.com/v2/usercollection/'
access_token = os.environ.get('OURA_TOKEN')
endpoints = ['daily_activity','daily_readiness', 'daily_sleep', 'daily_spo2', 'daily_stress']
start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

In [7]:
def test_oura_endpoint(endpoint, start_date, end_date):
    url = base_url + endpoint + '?start_date=' + start_date + '&end_date=' + end_date
    r = requests.get(url, headers={'Authorization': 'Bearer ' + access_token})
    return r.json()

In [8]:
# Test Each Endpoint
for endpoint in endpoints:
    data = test_oura_endpoint(endpoint, start_date, end_date)["data"]
    print(endpoint)
    print(f'Records: {len(data)}')
    print(f'Days: {[x["day"] for x in data]}')
    print(data)
    print()

daily_activity
Records: 1
Days: ['2024-02-02']
[{'id': '3321810a-23e1-4aeb-a6b1-d673bccb6308', 'class_5_min': '111111111111111111111111111111111111345333344444344444422223332322232322222222222322223222222223333223222222222222223232232222223322222322222222222223332334445555555443300003233233333333333333232333222233333322223332332234322222211111221111111111111111111111111111111111111111111111111111', 'score': 74, 'active_calories': 1401, 'average_met_minutes': 2.0625, 'contributors': {'meet_daily_targets': 100, 'move_every_hour': 100, 'recovery_time': 8, 'stay_active': 78, 'training_frequency': 100, 'training_volume': 100}, 'equivalent_walking_distance': 23389, 'high_activity_met_minutes': 461, 'high_activity_time': 2340, 'inactivity_alerts': 0, 'low_activity_met_minutes': 182, 'low_activity_time': 16920, 'medium_activity_met_minutes': 508, 'medium_activity_time': 6240, 'met': {'interval': 60.0, 'items': [1.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9

In [9]:
activity_columns = ['day', 'score', 'active_calories', 'contributors__meet_daily_targets', 'contributors__move_every_hour',
                    'contributors__recovery_time', 'contributors__stay_active', 'contributors__training_frequency', 
                    'contributors__training_volume', 'steps', 'target_calories', 'target_meters', 'total_calories', 
                    'day', 'high_activity_met_minutes', 'high_activity_time', 'medium_activity_met_minutes',
                    'medium_activity_time', 'low_activity_met_minutes', 'low_activity_time', 'sedentary_met_minutes',
                    'sedentary_time', 'resting_time', 'equivalent_walking_distance', 'inactivity_alerts']

readiness_columns = ['day', 'score', 'contributors__activity_balance', 'contributors__body_temperature', 'contributors__hrv_balance',
                    'contributors__previous_day_activity', 'contributors__previous_night', 'contributors__recovery_index', 
                    'contributors__resting_heart_rate', 'contributors__sleep_balance', 'temperature_deviation', 'temperature_trend_deviation']

sleep_columns = ['day', 'score', 'contributors__deep_sleep', 'contributors__efficiency', 'contributors__latency', 'contributors__rem_sleep',
                'contributors__restfulness', 'contributors__timing', 'contributors__total_sleep']

spo2_columns = ['day', 'spo2_percentage__average']

stress_columns = ['day', 'stress_high', 'recovery_high', 'day_summary']

oura_columns = [activity_columns, readiness_columns, sleep_columns, spo2_columns, stress_columns]

In [10]:
oura_columns_with_enpoints = dict(zip(endpoints, oura_columns))
oura_columns_with_enpoints

{'daily_activity': ['day',
  'score',
  'active_calories',
  'contributors__meet_daily_targets',
  'contributors__move_every_hour',
  'contributors__recovery_time',
  'contributors__stay_active',
  'contributors__training_frequency',
  'contributors__training_volume',
  'steps',
  'target_calories',
  'target_meters',
  'total_calories',
  'day',
  'high_activity_met_minutes',
  'high_activity_time',
  'medium_activity_met_minutes',
  'medium_activity_time',
  'low_activity_met_minutes',
  'low_activity_time',
  'sedentary_met_minutes',
  'sedentary_time',
  'resting_time',
  'equivalent_walking_distance',
  'inactivity_alerts'],
 'daily_readiness': ['day',
  'score',
  'contributors__activity_balance',
  'contributors__body_temperature',
  'contributors__hrv_balance',
  'contributors__previous_day_activity',
  'contributors__previous_night',
  'contributors__recovery_index',
  'contributors__resting_heart_rate',
  'contributors__sleep_balance',
  'temperature_deviation',
  'temperatur

In [11]:
def read_oura_endpoint(endpoint: str, start_date: str, end_date: str, columns: List[str]) -> pd.DataFrame:
    # Retrieve data from the endpoint
    json_data = test_oura_endpoint(endpoint, start_date, end_date)["data"]
    print(json_data)

    # Create a dictionary of the data
    df_dict_array = []
    if len(json_data) > 0:
        try:
            for day in json_data:
                df_dict = {}
                for col in columns:
                    if '__' in col:
                        parent = col.split('__')[0]
                        child = col.split('__')[1]
                        df_dict[col] = day[parent][child]
                    else:
                        df_dict[col] = day[col]
                df_dict_array.append(df_dict)
        except TypeError as e:
            if 'NoneType' in str(e):
                pass
            else:
                raise e
            
    df = pd.DataFrame(df_dict_array).drop_duplicates()

    # Convert the date column to a datetime object nd rename it to 'Date'
    df['day'] = pd.to_datetime(df['day']).dt.date
    df = df.rename(columns={'day': 'Day'})

    # Set the date column as the index
    df = df.set_index('Day')

    # Rename the columns
    # Remove the 'contributors__' prefix
    # Add the endpoint name as a prefix (excluding the daily part)
    # Change '_' to ' '
    # Uppercase the first letter of each word
    df = df.rename(columns=lambda x: x.replace('contributors__', '').replace('_', ' ').title())
    df = df.add_prefix(endpoint.replace('daily_', '').title() + ' ')
    # Change the 'Spo2 Spo2 Percentage  Average' column to 'SpO2 Average'
    if endpoint == 'daily_spo2':
        df = df.rename(columns={'Spo2 Spo2 Percentage  Average': 'SpO2 Percentage Average'})

    return df

In [12]:
start_date = '2024-01-11'
end_date = datetime.now().strftime('%Y-%m-%d')

df_dict = {}

for endpoint in endpoints:
    df_dict[endpoint] = read_oura_endpoint(endpoint, start_date, end_date, oura_columns_with_enpoints[endpoint])

df_dict

[{'id': 'c9695022-9689-45e3-83c3-d5d6ec7306fe', 'class_5_min': '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000223212222222222332222222222222212222222222222322222222222322344333433334433433213333332224343333323322233332222222222222223223333322222234223223222211111111111111111111111111111111111111111111111111111', 'score': 92, 'active_calories': 393, 'average_met_minutes': 1.34375, 'contributors': {'meet_daily_targets': 78, 'move_every_hour': 95, 'recovery_time': 100, 'stay_active': 86, 'training_frequency': 100, 'training_volume': 100}, 'equivalent_walking_distance': 7437, 'high_activity_met_minutes': 7, 'high_activity_time': 60, 'inactivity_alerts': 1, 'low_activity_met_minutes': 175, 'low_activity_time': 13860, 'medium_activity_met_minutes': 133, 'medium_activity_time': 2400, 'met': {'interval': 60.0, 'items': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 

{'daily_activity':             Activity Score  Activity Active Calories  \
 Day                                                    
 2024-01-11              92                       393   
 2024-01-12              93                       803   
 2024-01-13              95                       569   
 2024-01-14              98                      1918   
 2024-01-15              97                      1122   
 2024-01-16              94                       930   
 2024-01-17              91                       338   
 2024-01-18              96                       969   
 2024-01-19              98                       973   
 2024-01-20              94                       603   
 2024-01-21              97                       485   
 2024-01-22              88                       857   
 2024-01-23              87                       385   
 2024-01-24              83                       491   
 2024-01-25              80                       468   
 2024-01-26  

In [13]:
for df_key in df_dict.keys():
    print(df_key)
    print(df_dict[df_key].describe())
    print()

daily_activity
       Activity Score  Activity Active Calories  Activity Meet Daily Targets  \
count       23.000000                 23.000000                    23.000000   
mean        89.130435                883.695652                    83.695652   
std          7.225613                394.014586                    19.552807   
min         73.000000                338.000000                    43.000000   
25%         86.000000                530.000000                    69.000000   
50%         91.000000                910.000000                    95.000000   
75%         94.500000               1114.500000                   100.000000   
max         98.000000               1918.000000                   100.000000   

       Activity Move Every Hour  Activity Recovery Time  Activity Stay Active  \
count                 23.000000               23.000000             23.000000   
mean                  98.260870               89.000000             74.478261   
std                  

In [14]:
# Merge the dataframes
df = pd.concat(df_dict.values(), axis=1)
df

,Activity Score,Activity Active Calories,Activity Meet Daily Targets,Activity Move Every Hour,Activity Recovery Time,Activity Stay Active,Activity Training Frequency,Activity Training Volume,Activity Steps,Activity Target Calories,...,Sleep Efficiency,Sleep Latency,Sleep Rem Sleep,Sleep Restfulness,Sleep Timing,Sleep Total Sleep,SpO2 Percentage Average,Stress Stress High,Stress Recovery High,Stress Day Summary
Day,,,,,,,,,,,,,,,,,,,,,
2024-01-11,92.0,393.0,78.0,95.0,100.0,86.0,100.0,100.0,9548.0,450.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,None
2024-01-12,93.0,803.0,95.0,95.0,100.0,62.0,100.0,100.0,15938.0,600.0,...,99.0,72.0,84.0,86.0,91.0,74.0,99.498,0,0,None
2024-01-13,95.0,569.0,95.0,100.0,100.0,77.0,100.0,100.0,13340.0,450.0,...,96.0,59.0,58.0,88.0,59.0,78.0,99.265,0,0,None
2024-01-14,98.0,1918.0,95.0,100.0,100.0,96.0,100.0,100.0,19354.0,850.0,...,99.0,81.0,92.0,84.0,46.0,83.0,97.417,0,0,None
2024-01-15,97.0,1122.0,100.0,100.0,100.0,79.0,100.0,100.0,14844.0,700.0,...,97.0,83.0,96.0,90.0,53.0,78.0,98.392,0,0,None
2024-01-16,94.0,930.0,100.0,95.0,100.0,65.0,100.0,100.0,16211.0,700.0,...,93.0,72.0,45.0,88.0,83.0,61.0,96.178,0,0,None
2024-01-17,91.0,338.0,95.0,100.0,100.0,49.0,100.0,100.0,6897.0,700.0,...,98.0,83.0,95.0,91.0,88.0,80.0,96.582,2700,3600,normal
2024-01-18,96.0,969.0,100.0,95.0,100.0,77.0,100.0,100.0,14528.0,700.0,...,99.0,94.0,91.0,85.0,76.0,70.0,97.961,0,6300,restored
2024-01-19,98.0,973.0,100.0,95.0,100.0,87.0,100.0,100.0,17731.0,700.0,...,96.0,83.0,57.0,76.0,77.0,78.0,97.506,5400,4500,normal


## Adding Oura to Airtable

In [15]:
# Create a new Airtable base for Daily Oura Data
daily_oura_table_id = 'tblyljF7nLMRhIbwV'
daily_oura_airtable = AirtableData(personal_base_id, daily_oura_table_id, airtable_api_key)

In [16]:
print(daily_oura_airtable.get_data())

           Day  Activity Score  Activity Active Calories  \
0   2024-01-11            92.0                     393.0   
1   2024-01-28            87.0                    1107.0   
2   2024-01-30            91.0                    1408.0   
3   2024-01-17            91.0                     338.0   
4   2024-01-26            81.0                    1246.0   
5   2024-01-20            94.0                     603.0   
6   2024-01-27            89.0                    1171.0   
7   2024-01-29            87.0                     945.0   
8   2024-01-22            88.0                     857.0   
9   2024-01-25            80.0                     468.0   
10  2024-01-19            98.0                     973.0   
11  2024-01-18            96.0                     969.0   
12  2024-01-12            93.0                     803.0   
13  2024-02-01            73.0                     910.0   
14  2024-01-13            95.0                     569.0   
15  2024-02-02             NaN          

In [17]:
df_reset = df.reset_index()
df_reset['Day'] = df_reset['Day'].apply(lambda x: x.strftime('%Y-%m-%d'))  # Convert date to string
df_reset = df_reset.replace([np.inf, -np.inf, np.nan], None) # Replace inf and nan with None
df_reset


,Day,Activity Score,Activity Active Calories,Activity Meet Daily Targets,Activity Move Every Hour,Activity Recovery Time,Activity Stay Active,Activity Training Frequency,Activity Training Volume,Activity Steps,...,Sleep Efficiency,Sleep Latency,Sleep Rem Sleep,Sleep Restfulness,Sleep Timing,Sleep Total Sleep,SpO2 Percentage Average,Stress Stress High,Stress Recovery High,Stress Day Summary
0,2024-01-11,92.0,393.0,78.0,95.0,100.0,86.0,100.0,100.0,9548.0,...,None,None,None,None,None,None,None,0,0,None
1,2024-01-12,93.0,803.0,95.0,95.0,100.0,62.0,100.0,100.0,15938.0,...,99.0,72.0,84.0,86.0,91.0,74.0,99.498,0,0,None
2,2024-01-13,95.0,569.0,95.0,100.0,100.0,77.0,100.0,100.0,13340.0,...,96.0,59.0,58.0,88.0,59.0,78.0,99.265,0,0,None
3,2024-01-14,98.0,1918.0,95.0,100.0,100.0,96.0,100.0,100.0,19354.0,...,99.0,81.0,92.0,84.0,46.0,83.0,97.417,0,0,None
4,2024-01-15,97.0,1122.0,100.0,100.0,100.0,79.0,100.0,100.0,14844.0,...,97.0,83.0,96.0,90.0,53.0,78.0,98.392,0,0,None
5,2024-01-16,94.0,930.0,100.0,95.0,100.0,65.0,100.0,100.0,16211.0,...,93.0,72.0,45.0,88.0,83.0,61.0,96.178,0,0,None
6,2024-01-17,91.0,338.0,95.0,100.0,100.0,49.0,100.0,100.0,6897.0,...,98.0,83.0,95.0,91.0,88.0,80.0,96.582,2700,3600,normal
7,2024-01-18,96.0,969.0,100.0,95.0,100.0,77.0,100.0,100.0,14528.0,...,99.0,94.0,91.0,85.0,76.0,70.0,97.961,0,6300,restored
8,2024-01-19,98.0,973.0,100.0,95.0,100.0,87.0,100.0,100.0,17731.0,...,96.0,83.0,57.0,76.0,77.0,78.0,97.506,5400,4500,normal
9,2024-01-20,94.0,603.0,95.0,100.0,100.0,66.0,100.0,100.0,11064.0,...,95.0,94.0,50.0,78.0,80.0,57.0,95.618,30600,0,stressful


In [18]:
daily_oura_airtable.upsert_data(df_reset, id_field='Day', date_type='date')